::: {.content-hidden when-format="html"}

```{=tex}
\DeclareMathOperator*{\argmin}{arg\,min}
\newcommand{\bs}[1]{\boldsymbol{#1}}
\newcommand{\mbf}[1]{\mathbf{#1}}
```

:::

# Introduction

Hyperparameter optimization is a critical aspect of machine learning, where the objective is to discover the ideal configuration settings, or "hyperparameters", for a given model. While standard parameters are adjusted during the training process based on data, hyperparameters, such as learning rates or regularization terms, are set prior to this phase. These can include the weight decay parameter, the number of hidden layers, etc. The goal is to find hyperparameters that result in the best model performance, often measured by validation accuracy or loss.

The standard technique for hyperparamter optimization (HO) is grid search, where every possible combination of hyperparameters is tested. Because of the vast potential combinations of hyperparameters, you random search is often employed where RANDOM. The problem with either of these techniques is that they require you to train multiple models that will later be discarded [@mlodozeniec2023hyperparameter, p. 1]. Moreover, these methods are not scalable to cases with many hyperparameters [@lorraine2020optimizing] [@maclaurin2015gradient]

It would be preferable if we could update hyperparameters in the same way we update model parameters, i.e., through the use of gradient-based methods. Similar to updates to parameters, and differently from grid search methods, we hope that we can make use of previous evaluation to "make an informed decision of the next iterate" [@pedregosa2016hyperparameter]. In this post, I will illustrate the application of gradient-based HO methods to a simple neural network to optimize the weight decay hyperparameter. I thought this would be a good test case to get more familiar with PyTorch.

# Gradient Computation

The first hurdle to overcome in gradient HO is the fact that you cannot optimize hyperparameter to minimize the same loss function as the parameters; in the example of weight decay, this would always lead to preferring models with no penalization [@pedregosa2016hyperparameter]. As a result, we need to use cross-validation or a holdout validation set to optimize a criterion on unseen data. We use a validation set and construct HO as a two-level optimization problem as in @pedregosa2016hyperparameter and @lorraine2020optimizing. Let $\mathcal{L}_T$ and $\mathcal{L}_V$ be the training and validation loss, respectively. Both of these depend on the hyperparamters $\boldsymbol\lambda$ and model parameters $\mathbf{w}$, whether directly or indircetly (more on this later). First, we train the model and update the model parameters for a given value of $\boldsymbol\lambda$. Then we update the hyperparameters:

$$ 
\begin{align}
\boldsymbol\lambda^\ast &\equiv \mathop{\mathrm{arg\,min}}\mathcal{L}_V^\ast(\boldsymbol\lambda), \quad \textrm{where} \\
\mathcal{L}_V^\ast(\boldsymbol\lambda) &\equiv \mathcal{L}_V(\boldsymbol\lambda, \mathbf{w}^\ast(\boldsymbol\lambda)) \, \textrm{and} \, \mathbf{w}^\ast(\boldsymbol\lambda) \equiv \mathop{\mathrm{arg\,min}}\mathcal{L}_T(\boldsymbol\lambda, \mathbf{w})
\end{align}
$$

The second issue is more difficult to handle. The validation loss can depend on the hyperparameters both directly but also indirectly through the optimal weights $w^\ast$. We have to calculate how the optimal weights change as the hyperparameters change [@lorraine2020optimizing].

Following [@lorraine2020optimizing], the training loss is an implicit function of $\boldsymbol\lambda$. Under certain regularity conditions, the optimal values of the parameters are given by the equation (implicity a function of $\boldsymbol{\lambda}$):

$$
\dfrac{\partial \mathcal{L}_T}{\partial \mathbf{w}} \Bigg|_{\boldsymbol{\lambda},\mathbf{w}^\ast(\boldsymbol{\lambda})} = 0
$$

The implicit equation with respect to $\boldsymbol{\lambda}$ is then:

$$
\dfrac{\partial^2 \mathcal{L}_T}{\partial \mathbf{w} \partial \boldsymbol{\lambda}^T} + \dfrac{\partial^2 \mathcal{L}_T}{\partial \mathbf{w}\partial \mathbf{w}^T}\dfrac{\partial \mathbf{w}^\ast}{\partial \boldsymbol{\lambda}} = 0
$$

Assuming that the derivative of $\dfrac{\partial^2 \mathcal{L}_T}{\partial \mathbf{w}\partial \mathbf{w}^T}$ exists, we can decompose the validation gradient with respect to $\boldsymbol{\lambda}$ as:

$$
\begin{align*}
\dfrac{\partial \mathcal{L}_V^\ast(\boldsymbol\lambda, \mathbf{w}^\ast(\boldsymbol{\lambda}))}{\partial \boldsymbol{\lambda}} &= \dfrac{\partial \mathcal{L}_V(\boldsymbol\lambda, \mathbf{w}^\ast(\boldsymbol{\lambda}))}{\partial \boldsymbol{\lambda}} + \dfrac{\partial \mathcal{L}_V(\boldsymbol\lambda, \mathbf{w}^\ast(\boldsymbol{\lambda}))}{\partial \mathbf{w}^\ast}  \dfrac{\partial \mathbf{w}^\ast}{\partial \boldsymbol{\lambda}}\\
&= \dfrac{\partial \mathcal{L}_V(\boldsymbol\lambda, \mathbf{w}^\ast(\boldsymbol{\lambda}))}{\partial \boldsymbol{\lambda}} - \left(\dfrac{\partial^2 \mathcal{L}_T}{\partial \mathbf{w} \partial \boldsymbol{\lambda}^T}\right)^T \left(\dfrac{\partial^2 \mathcal{L}_T}{\partial \mathbf{w}\partial \mathbf{w}^T}\right)^{-1}\dfrac{\partial \mathcal{L}_V(\boldsymbol\lambda, \mathbf{w}^\ast(\boldsymbol{\lambda}))}{\partial \mathbf{w}^\ast}
\end{align*}
$$

So, by placing regularity conditions on the training loss, we can calculate the gradient of the validation loss with respect to the hyperparameters and use that to iteratively update these hyperparameters.

## Computation

The issue noew is purely computational: how can we quickly calculate the Hessian matrix and its inverse in the above equation. The two papers both lay out how to calculate an approximate solution to the inverse Hessian-vector product:

[@pedregosa2016hyperparameter] suggest using the conjugate gradient method to find

$$
\left(\dfrac{\partial^2 \mathcal{L}_T}{\partial \mathbf{w}\partial \mathbf{w}^T}\right)^{-1}\dfrac{\partial \mathcal{L}_V(\boldsymbol\lambda, \mathbf{w}^\ast(\boldsymbol{\lambda}))}{\partial \mathbf{w}^\ast}
$$

In other words, solve:

$$
\dfrac{\partial^2 \mathcal{L}_T}{\partial \mathbf{w}\partial \mathbf{w}^T} \mathbf{x} = \dfrac{\partial \mathcal{L}_V(\boldsymbol\lambda, \mathbf{w}^\ast(\boldsymbol{\lambda}))}{\partial \mathbf{w}^\ast} 
$$ HOAG only requires Hessian vector product so never need ot instantiate Hessian (can cite both HOAG and Lorraine paper)

[@lorraine2020optimizing] instead proposes the calcualtion of the first several terms of the Neumann series:

$$
\left(\dfrac{\partial^2 \mathcal{L}_T}{\partial \mathbf{w}\partial \mathbf{w}^T}\right)^{-1} = \lim_{i \to \infty} \displaystyle\sum_{j=0}^i\left(I - \dfrac{\partial^2 \mathcal{L}_T}{\partial \mathbf{w}\partial \mathbf{w}^T}\right)^j
$$

The propose a clever algorithm that calculates the terms of this series recursively and do not require instantiating the second derivative matrix, only the vector-Hessian products.

$$
\begin{align*}
v_0 &= \dfrac{\partial \mathcal{L}_V}{\partial \mathbf{w}} \\
v_1 &= v_0 - v_0 \dfrac{\partial^2 \mathcal{L}_T}{\partial \mathbf{w} \partial \mathbf{w}^T} = v_0\left(I - \dfrac{\partial^2 \mathcal{L}_T}{\partial \mathbf{w} \partial \mathbf{w}^T}\right) \\
v_2 &= v_1 - v_1 \dfrac{\partial^2 \mathcal{L}_T}{\partial \mathbf{w} \partial \mathbf{w}^T} = v_1\left(I - \dfrac{\partial^2 \mathcal{L}_T}{\partial \mathbf{w} \partial \mathbf{w}^T}\right) = v_0\left(I - \dfrac{\partial^2 \mathcal{L}_T}{\partial \mathbf{w} \partial \mathbf{w}^T}\right)^2 \\
&\ldots
\end{align*}
$$ This is simple to implement in PyTorch because of the ease in calculating vector-Jacobian products.


In [ ]:
import torch
# from torch.autograd.functional import hessian
# 
# def f(x):
#     return x.pow(3).sum()
# 
# def neumann_series(v, f, I=50):
#     v = v.detach().clone()
#     p = v.detach().clone()
#     for i in range(I):
#         tmp_v = torch.autograd.grad(f, inputs, grad_outputs=v, retain_graph=True)
#         tmp_v = torch.cat([grad.view(-1) for grad in tmp_v])
#         v = v - tmp_v
#         p = p + v
#     return p
# 
# v = torch.tensor([1, 1], dtype=torch.float32)
# inputs = torch.tensor([0.1, 0.1], requires_grad=True)
# hess = hessian(f, inputs)
# invhess = torch.linalg.inv(hess)
# 
# v @ invhess
# 
# y = f(inputs)
# grad_y = torch.autograd.grad(y, inputs, create_graph=True)
# 
# v = v.detach().clone()
# p = v.detach().clone()
# for i in range(50):
#     tmp_v = torch.autograd.grad(grad_y, inputs, grad_outputs=v, retain_graph=True)
#     tmp_v = torch.cat([grad.view(-1) for grad in tmp_v])
#     v = v - tmp_v
#     p = p + v

In practice, most of the contrived examples I created did not satisfy this sufficient condition and the Neumann series rarely converged.

# Results

The full code can be found in the Github [repository](https://github.com/mdk31/hyperparameter-optimization)

Figure @fig-grid shows


In [ ]:
#| label: fig-grid
#| fig-cap: Validation error trajectory by lambda
3 - 2

In [ ]:
#| label: fig-hoag
#| fig-cap: Validation error trajectory by starting lambda
3 - 2

The rapid convergence of the $\lambda$ to the low $\lambda$ values (and the consequent drop in the validation error) is striking. The actual [@pedregosa2016hyperparameter] has a complex set of tolerances that vary over update epochs to ensure theroetical convergence of the HOAG algorithm as a whole. Even without that, it performs well with fixed tolerances.

## Drawbacks

The main drawback to this methodology is that it requires the Hessian to be invertible, even if you don't have to calculate it exactly. In practice, the Hessian is rarely invertible in deep learning problems [@sagun2016eigenvalues].

The conjugate gradient descent method I use is also rather slow; the HOAG method took about 10 minutes to run.